In [1]:
import math as m
import numpy as np
import casadi as ca
import matplotlib.pyplot as plt
import matplotlib

SMALL_SIZE = 15
matplotlib.rc('font', size=SMALL_SIZE)
matplotlib.rc('axes', titlesize=SMALL_SIZE, labelsize=SMALL_SIZE)

In [ ]:
# Exact parameters fit for each dose
doses_data_all = np.array([0.032, 0.1, 0.32, 1, 3.2]);
doses_data_sub = np.array([0.032, 0.32, 3.2]);

# Fits across all 5 doses
r_S_fit_all = np.array([0.0532, 0.0630, 0.0722, 0.0539, 0.0880])
d_S_fit_all = np.array([0.1855, 0.3394, 0.4012, 0.2862, 0.6924])
alpha_fit_all = np.array([0.2520, 0.2155, 0.1963, 0.1378, 0.1048])
r_R_fit_all = np.array([0.0532, 0.0519, 0.0415, 0.0190, 0.0070])
d_R_fit_all = np.array([0.0751, 0.0819, 0.0699, 0.0339, 0.0137])

# Fits only at doses_data_sub
r_S_fit_sub = np.array([0.0532, 0.0722, 0.0880])
d_S_fit_sub = np.array([0.1855, 0.4012, 0.6924])
alpha_fit_sub = np.array([0.2520, 0.1963, 0.1048])
r_R_fit_sub = np.array([0.0532, 0.0415, 0.0070])
d_R_fit_sub = np.array([0.0751, 0.0699, 0.0137])

# Plot the data fits, the piecewise linear interpolated values (using doses_data_sub), and the fit polynomial
# Recall that we interpolate on the log scale
r_S_lin_inter = np.interp(np.log10(doses_data_all), np.log10(doses_data_sub), r_S_fit_sub)
d_S_lin_inter = np.interp(np.log10(doses_data_all), np.log10(doses_data_sub), d_S_fit_sub)
alpha_lin_inter = np.interp(np.log10(doses_data_all), np.log10(doses_data_sub), alpha_fit_sub)
r_R_lin_inter = np.interp(np.log10(doses_data_all), np.log10(doses_data_sub), r_R_fit_sub)
d_R_lin_inter = np.interp(np.log10(doses_data_all), np.log10(doses_data_sub), d_R_fit_sub)

# Now evaluate the polynomial
# First define helper functions to define the polynomial
def coeff_poly_fit(x_data, y_data, deg=None):
    # Find coefficients that fit (x_data, y_data) of a given degree deg
    # x_data:  x coordinates of data (numpy array)
    # y_data:  y coordinates of data (numpy array)
    # deg:  degree of polynomial to fit (default is interpolation)
    # x_data and y_data must have equal length
    # Returns coefficients for corresponding polynomial

    # No degree provided, then use interpolation
    if deg is None:
        deg = len(x_data) - 1
    fit = np.polynomial.polynomial.Polynomial.fit(x_data, y_data, deg)
    coeff = fit.convert().coef  # Get the coefficients
    return coeff[::-1]          # Return coefficients in correct order for polyval

def poly_fun(x, coeffs):
    # Evaluate the polyomial at x with given coefficients
    # x:  point of evaluation
    # coeffs:  coefficients defining polyomial (starts at highest degree)
    if type(x) is ca.casadi.MX:
        coeffs_casadi = ca.MX(coeffs)
        return ca.polyval(coeffs_casadi, x)
    else:
        return np.polyval(coeffs, x)

# For plotting the fit polynomial
doses_plot = np.linspace(0.032, 3.2, 1000)

# Pick a degree and fit
deg_r_S = 3
r_S_coeffs = coeff_poly_fit(np.log10(doses_data_all), r_S_lin_inter, deg=deg_r_S)
r_S_polynomial_plot = poly_fun(np.log10(doses_plot), r_S_coeffs)

deg_d_S = 3
d_S_coeffs = coeff_poly_fit(np.log10(doses_data_all), d_S_lin_inter, deg=deg_d_S)
d_S_polynomial_plot = poly_fun(np.log10(doses_plot), d_S_coeffs)

deg_alpha = 3
alpha_coeffs = coeff_poly_fit(np.log10(doses_data_all), alpha_lin_inter, deg=deg_alpha)
alpha_polynomial_plot = poly_fun(np.log10(doses_plot), alpha_coeffs)

deg_r_R = 3
r_R_coeffs = coeff_poly_fit(np.log10(doses_data_all), r_R_lin_inter, deg=deg_r_R)
r_R_polynomial_plot = poly_fun(np.log10(doses_plot), r_R_coeffs)

deg_d_R = 3
d_R_coeffs = coeff_poly_fit(np.log10(doses_data_all), d_R_lin_inter, deg=deg_d_R)
d_R_polynomial_plot = poly_fun(np.log10(doses_plot), d_R_coeffs)

# Reminder of what degree using in the fit
r_S_label_str_poly = 'polynomial approximation (degree = ' + str(deg_r_S) + ')'
d_S_label_str_poly = 'polynomial approximation (degree = ' + str(deg_d_S) + ')'
alpha_label_str_poly = 'polynomial approximation (degree = ' + str(deg_alpha) + ')'
r_R_label_str_poly = 'polynomial approximation (degree = ' + str(deg_r_R) + ')'
d_R_label_str_poly = 'polynomial approximation (degree = ' + str(deg_d_R) + ')'

fig = plt.figure(figsize=(25,25))
fig.subplots_adjust(hspace=0.5)

plt.subplot(5, 2, 1)
plt.semilogx(doses_data_sub, r_S_fit_sub, marker='o', linestyle='None', color='red', markersize=10, label='fit')
plt.semilogx(doses_data_all, r_S_lin_inter , marker='x', linestyle='-', color='blue', markersize=10, label='linearly interpolate (log scale)')
plt.semilogx(doses_plot, r_S_polynomial_plot, marker='None', linestyle='--', color='black', label=r_S_label_str_poly)
plt.legend(loc='best')
plt.xlabel('dose ($\mu M$)')
plt.title('$r_{S}$ (log scale)')
plt.xticks(ticks=doses_data_all, labels=doses_data_all)

plt.subplot(5, 2, 2)
plt.plot(doses_data_sub, r_S_fit_sub, marker='o', linestyle='None', color='red', markersize=10, label='fits')
plt.plot(doses_data_all, r_S_lin_inter , marker='x', linestyle='-', color='blue', markersize=10, label='linear interpolant')
plt.plot(doses_plot, r_S_polynomial_plot, marker='None', linestyle='--', color='black', label=r_S_label_str_poly)
#plt.legend(loc='best')
plt.xlabel('dose ($\mu M$)')
plt.title('$r_{S}$ (linear scale)')
plt.xticks(ticks=doses_data_all, labels=doses_data_all, fontsize=8)

plt.subplot(5, 2, 3)
plt.semilogx(doses_data_sub, d_S_fit_sub, marker='o', linestyle='None', color='red', markersize=10, label='fit')
plt.semilogx(doses_data_all, d_S_lin_inter , marker='x', linestyle='-', color='blue', markersize=10, label='linear interpolantly (log scale)')
plt.semilogx(doses_plot, d_S_polynomial_plot, marker='None', linestyle='--', color='black', label=d_S_label_str_poly)
plt.legend(loc='best')
plt.xlabel('dose ($\mu M$)')
plt.title('$d_{S}$ (log scale)')
plt.xticks(ticks=doses_data_all, labels=doses_data_all)

plt.subplot(5, 2, 4)
plt.plot(doses_data_sub, d_S_fit_sub, marker='o', linestyle='None', color='red', markersize=10, label='fits')
plt.plot(doses_data_all, d_S_lin_inter , marker='x', linestyle='-', color='blue', markersize=10, label='linear interpolant')
plt.plot(doses_plot, d_S_polynomial_plot, marker='None', linestyle='--', color='black', label=d_S_label_str_poly)
#plt.legend(loc='best')
plt.xlabel('dose ($\mu M$)')
plt.title('$d_{S}$ (linear scale)')
plt.xticks(ticks=doses_data_all, labels=doses_data_all, fontsize=8)

plt.subplot(5, 2, 5)
plt.semilogx(doses_data_sub, alpha_fit_sub, marker='o', linestyle='None', color='red', markersize=10, label='fit')
plt.semilogx(doses_data_all, alpha_lin_inter , marker='x', linestyle='-', color='blue', markersize=10, label='linearly interpolate (log scale)')
plt.semilogx(doses_plot, alpha_polynomial_plot, marker='None', linestyle='--', color='black', label=alpha_label_str_poly)
plt.legend(loc='best')
plt.xlabel('dose ($\mu M$)')
plt.title(r'$\alpha$ (log scale)')
plt.xticks(ticks=doses_data_all, labels=doses_data_all)

plt.subplot(5, 2, 6)
plt.plot(doses_data_sub, alpha_fit_sub, marker='o', linestyle='None', color='red', markersize=10, label='fits')
plt.plot(doses_data_all, alpha_lin_inter , marker='x', linestyle='-', color='blue', markersize=10, label='linear interpolate (log scale)')
plt.plot(doses_plot, alpha_polynomial_plot, marker='None', linestyle='--', color='black', label=alpha_label_str_poly)
#plt.legend(loc='best')
plt.xlabel('dose ($\mu M$)')
plt.title(r'$\alpha$ (linear scale)')
plt.xticks(ticks=doses_data_all, labels=doses_data_all, fontsize=8)

plt.subplot(5, 2, 7)
plt.semilogx(doses_data_sub, r_R_fit_sub, marker='o', linestyle='None', color='red', markersize=10, label='fit')
plt.semilogx(doses_data_all, r_R_lin_inter , marker='x', linestyle='-', color='blue', markersize=10, label='linearly interpolate (log scale)')
plt.semilogx(doses_plot, r_R_polynomial_plot, marker='None', linestyle='--', color='black', label=r_R_label_str_poly)
plt.legend(loc='best')
plt.xlabel('dose ($\mu M$)')
plt.title('$r_{R}$ (log scale)')
plt.xticks(ticks=doses_data_all, labels=doses_data_all)

plt.subplot(5, 2, 8)
plt.plot(doses_data_sub, r_R_fit_sub, marker='o', linestyle='None', color='red', markersize=10, label='fits')
plt.plot(doses_data_all, r_R_lin_inter , marker='x', linestyle='-', color='blue', markersize=10, label='linear interpolant')
plt.plot(doses_plot, r_R_polynomial_plot, marker='None', linestyle='--', color='black', label=r_R_label_str_poly)
#plt.legend(loc='best')
plt.xlabel('dose ($\mu M$)')
plt.title('$r_{R}$ (linear scale)')
plt.xticks(ticks=doses_data_all, labels=doses_data_all, fontsize=8)

plt.subplot(5, 2, 9)
plt.semilogx(doses_data_sub, d_R_fit_sub, marker='o', linestyle='None', color='red', markersize=10, label='fit')
plt.semilogx(doses_data_all, d_R_lin_inter , marker='x', linestyle='-', color='blue', markersize=10, label='linearly interpolate (log scale)')
plt.semilogx(doses_plot, d_R_polynomial_plot, marker='None', linestyle='--', color='black', label=d_R_label_str_poly)
plt.legend(loc='best')
plt.xlabel('dose ($\mu M$)')
plt.title('$d_{R}$ (log scale)')
plt.xticks(ticks=doses_data_all, labels=doses_data_all)

plt.subplot(5, 2, 10)
plt.plot(doses_data_sub, d_R_fit_sub, marker='o', linestyle='None', color='red', markersize=10, label='fits')
plt.plot(doses_data_all, d_R_lin_inter , marker='x', linestyle='-', color='blue', markersize=10, label='linear interpolant')
plt.plot(doses_plot, d_R_polynomial_plot, marker='None', linestyle='--', color='black', label=d_R_label_str_poly)
#plt.legend(loc='best')
plt.xlabel('dose ($\mu M$)')
plt.title('$d_{R}$ data, linear interpolant, and polynomial approximation (linear scale)')
plt.xticks(ticks=doses_data_all, labels=doses_data_all, fontsize=8)

#plt.savefig('rate_smoothing.png')

plt.show()